## Swarm Simulation 

This is for testing propouses

In [1]:
import os
import subprocess
from pymavlink import mavutil

# Custom modules
from vehicle_logic import VehicleLogic
from helpers.change_coordinates import heading_to_yaw,find_spawns
from helpers.gazebo_world import update_world

## Kill all related process

In [2]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

# Choose Simulator

In [3]:
simulator ='gazebo' #'QGroundControl' # 

# Choose Initial Position

In [4]:
offsets = [ #east, north, up, heading
    (5, 5, 0, 90),
    (10, 0, 0, 45),
    (-5, -5, 0, 225),
    (-15, 0, 0, 0),
    (0, -20, 0, 0),
]
n_uavs = len(offsets)

                # lat,long,alt,heading
if simulator == 'QGroundControl':
    home_position=(-35.3633245,149.1652241,0,90)
    spawns=find_spawns(home_position, offsets)

## Launch Simulator

In [5]:
if simulator == 'QGroundControl':
    sim_path = os.path.expanduser("~/QGroundControl.AppImage")
    sim_cmd =[sim_path]
elif simulator == 'gazebo':
    # Convert to Gazebo format (name, x, y, z, roll, pitch, yaw)
    drones = [(f"drone{i+1}", east, north, up, 0, 0, heading_to_yaw(heading)) for i, (east, north, up, heading) in enumerate(offsets)]
    world_path = os.path.expanduser("~/ardupilot_gazebo/worlds/multiple_runway.world")
    updated_world_path = update_world(drones,world_path)
    sim_cmd = ["gazebo", "--verbose", updated_world_path] 

simulator_process = subprocess.Popen(
                    sim_cmd,
                    stdout=subprocess.DEVNULL,  # Suppress standard output
                    stderr=subprocess.DEVNULL,  # Suppress error output
                    shell=False  # Ensure safety when passing arguments
                    )

## Create Plan

In [6]:
plan=['check_prearm','check_pos_est','mode_stabilize','mode_guided','arm','takeoff','fly','land']
uavs_arg = []
for i in range(n_uavs):
    altitude = 5
    distance = 10+3*i
    wps=[(0, 0, -altitude), #takeoff point
        (0,distance, -altitude),
        (distance, distance, -altitude),
        (distance, 0, -altitude),
        (0, 0, -altitude)]
    uav_arg={'waypoints':wps,
             'plan': plan,
             'wp_margin': 0.5}
    uavs_arg.append(uav_arg)

# Launch Vehicle

In [7]:
vehicle_path = os.path.expanduser("~/ardupilot/Tools/autotest/sim_vehicle.py")

In [8]:
for i in range(n_uavs):
    vehicle_cmd = f"python3 {vehicle_path} -v ArduCopter -I{i} --sysid {i+1} --no-rebuild"
    if simulator == 'QGroundControl':
        spawn=','.join(map(str, spawns[i]))
        vehicle_cmd += f" --custom-location={spawn}"
    elif simulator == 'gazebo':
        vehicle_cmd += " -f gazebo-iris"
    subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

## Create UavLogic

In [9]:
uavs=[]
for i,uav_arg in enumerate(uavs_arg):
    connection = mavutil.mavlink_connection(f'udp:127.0.0.1:{14551+10*i}')
    connection.wait_heartbeat()
    uav=VehicleLogic(connection,**uav_arg)
    uavs.append(uav)

vehicle 1 created
vehicle 2 created
vehicle 3 created
vehicle 4 created
vehicle 5 created


## Execute Plan

In [10]:
mission_incomplete = n_uavs*[True]
while any(mission_incomplete):
    for i,uav in enumerate(uavs):
        mission_incomplete[i] = uav.act_plan()

vehicle 1: action land is done
vehicle 2: action land is done
vehicle 3: action land is done
vehicle 4: action land is done
vehicle 5: action land is done
vehicle 1: action check_prearm is done
vehicle 2: action check_prearm is done
vehicle 3: action check_prearm is done
vehicle 4: action check_prearm is done
vehicle 5: action check_prearm is done
vehicle 1: action check_pos_est is done
vehicle 4: action check_pos_est is done
vehicle 1: action mode_stabilize is done
vehicle 2: action check_pos_est is done
vehicle 3: action check_pos_est is done
vehicle 4: action mode_stabilize is done
vehicle 5: action check_pos_est is done
vehicle 1: action mode_guided is done
vehicle 2: action mode_stabilize is done
vehicle 3: action mode_stabilize is done
vehicle 4: action mode_guided is done
vehicle 5: action mode_stabilize is done
vehicle 1: action arm is done
vehicle 2: action mode_guided is done
vehicle 3: action mode_guided is done
vehicle 4: action arm is done
vehicle 5: action mode_guided is 

KeyboardInterrupt: 

## Kill all related process

In [ ]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")